# Week 2: Latent Factor-Based Recommender Systems

Last week we went over some basics of Recommender Systems for similarity based recommendations. In this notebook we will learn the basics of Latent Factor Models, as well as how to implement them.

## Part 1: Setting up the Data

This week we will be using another amazon review dataset, this time the dataset is about Watches. 
https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz

In [1]:
import gzip
from collections import defaultdict
import scipy
import scipy.optimize
import numpy
import random

path = "/home/ankit/Downloads/amazon_reviews_us_Digital_Video_Games_v1.tsv.gz"

f = gzip.open(path, 'rt', encoding="utf8")

header = f.readline()
header = header.strip().split('\t')
dataset = []

for line in f:
    fields = line.strip().split('\t')
    d = dict(zip(header, fields))
    d['star_rating'] = int(d['star_rating'])
    d['helpful_votes'] = int(d['helpful_votes'])
    d['total_votes'] = int(d['total_votes'])
    dataset.append(d)

In [2]:
dataset[0]

{'marketplace': 'US',
 'customer_id': '21269168',
 'review_id': 'RSH1OZ87OYK92',
 'product_id': 'B013PURRZW',
 'product_parent': '603406193',
 'product_title': 'Madden NFL 16 - Xbox One Digital Code',
 'product_category': 'Digital_Video_Games',
 'star_rating': 2,
 'helpful_votes': 2,
 'total_votes': 3,
 'vine': 'N',
 'verified_purchase': 'N',
 'review_headline': 'A slight improvement from last year.',
 'review_body': "I keep buying madden every year hoping they get back to football. This years version is a little better than last years -- but that's not saying much.The game looks great. The only thing wrong with the animation, is the way the players are always tripping on each other.<br /><br />The gameplay is still slowed down by the bloated pre-play controls. What used to take two buttons is now a giant PITA to get done before an opponent snaps the ball or the play clock runs out.<br /><br />The turbo button is back, but the player movement is still slow and awkward. If you liked las

In [4]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)

for d in dataset:
    user,item = d['customer_id'], d['product_id']
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)

## Part 2: Simple Latent Factor-Based Recomender

Here we'll use gradient descent to implement a machine-learning-based recommender (a latent-factor model).

This is a fairly difficult exercise, but brings together many of the ideas we've seen previously in this class, especially regarding gradient descent. This will be a relatively light notebook given this case, but you will need to know how to do this __on your own__ for your capstone project!

First, we build some utility data structures to store the variables of our model (alpha, userBiases, and itemBiases)

In [11]:
#Getting the respective lengths of our dataset and dictionaries
N = len(dataset)
nUsers = len(reviewsPerUser)
nItems = len(reviewsPerItem)

#Getting a list of keys
users = list(reviewsPerUser.keys())
items = list(reviewsPerItem.keys())

#This is equivalent to our Rating Mean from week 1
alpha = sum([d['star_rating'] for d in dataset]) / N

#Create another two defaultdict's, this time being float types because they are prediction based
userBiases = defaultdict(float)
itemBiases = defaultdict(float)

#Can't forget our MSE function
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

The actual prediction function of our model is simple: Just predict using a global offset (alpha), a user offset (beta_u in the slides), and an item offset (beta_i)

In [12]:
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item]

We'll use another library in this example to perform gradient descent. This library requires that we pass it a "flat" parameter vector (theta) containing all of our parameters. This utility function just converts between a flat feature vector, and our model parameters, i.e., it "unpacks" theta into our offset and bias parameters.

In [13]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))

The "cost" function is the function we are trying to optimize. Again this is a requirement of the gradient descent library we'll use. In this case, we're just computing the (regularized) MSE of a particular solution (theta), and returning the cost.

In [14]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d['customer_id'], d['product_id']) for d in dataset]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost

The derivative function is the most difficult to implement, but follows the definitions of the derivatives for this model as given in the lectures. This step could be avoided if using a gradient descent implementation based on (e.g.) Tensorflow.

In [15]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(dataset)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for d in dataset:
        u,i = d['customer_id'], d['product_id']
        pred = prediction(u, i)
        diff = pred - d['star_rating']
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    return numpy.array(dtheta)

Compute the MSE of a trivial baseline (always predicting the mean) for comparison:

In [16]:
alwaysPredictMean = [alpha for d in dataset]
labels = [d['star_rating'] for d in dataset]

MSE(alwaysPredictMean, labels) #Should be 1.6725...

2.371535478415058

Finally, we can run gradient descent. This particular gradient descent library takes as inputs (1) Our cost function (implemented above); (2) Initial parameter values; (3) Our derivative function; and (4) Any additional arguments to be passed to the cost function (in this case the labels and the regularization strength).

In [17]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),
                             derivative, args = (labels, 0.001))

MSE = 2.371535478415058
MSE = 2.2452414775741834
MSE = 3.513216788711963
MSE = 2.211284736177429
MSE = 2.128272434499708
MSE = 2.123784643013115
MSE = 2.108232343877302
MSE = 2.050662177391699
MSE = 2.0084242729426505
MSE = 1.9638605236076283
MSE = 1.9496659693082132
MSE = 1.9526044752581007
MSE = 1.949979803237113
MSE = 1.9471775209120896
MSE = 1.941671559561535
MSE = 1.9371723194881387
MSE = 1.9372094011180512
MSE = 1.937117203697303
MSE = 1.938712822961306
MSE = 1.937105812716197
MSE = 1.936771867225814
MSE = 1.9365826799592682
MSE = 1.9364439086814305
MSE = 1.9364491895852591
MSE = 1.9363337231331876
MSE = 1.9363747332089618
MSE = 1.936336921335685
MSE = 1.9361886799365242
MSE = 1.936029431269103
MSE = 1.9361349519560318
MSE = 1.9360882413716394
MSE = 1.9360580364176427
MSE = 1.9360133449333181
MSE = 1.9360450849300763
MSE = 1.9360468678370508
MSE = 1.9360496055736482
MSE = 1.9360440361079956
MSE = 1.93604142122933
MSE = 1.9360401177354334


(array([ 3.67299564e+00, -3.01669105e-03,  4.11425343e-03, ...,
        -4.53449134e-03, -1.15367480e-02,  8.99700060e-03]),
 2.019037845378189,
 {'grad': array([-6.09040248e-06, -6.16486043e-09,  1.82678181e-09, ...,
         -4.49890510e-10,  1.15862737e-08, -7.80521295e-09]),
  'task': b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'funcalls': 39,
  'nit': 32,
  'warnflag': 0})

## Part 3: Complete Latent Factor Model



For each user and item we now have a low dimensional descriptor (which represents a user's preferences), of dimension K.

In [18]:
userBiases = defaultdict(float)
itemBiases = defaultdict(float)
userGamma = {}
itemGamma = {}

K = 2

In [19]:
for u in reviewsPerUser:
    userGamma[u] = [random.random() * 0.1 - 0.05 for k in range(K)]
    
for i in reviewsPerItem:
    itemGamma[i] = [random.random() * 0.1 - 0.05 for k in range(K)]

Again we must implement an "unpack" function. This is the same as before, though has some additional terms.

In [20]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    global userGamma
    global itemGamma
    index = 0
    alpha = theta[index]
    index += 1
    userBiases = dict(zip(users, theta[index:index+nUsers]))
    index += nUsers
    itemBiases = dict(zip(items, theta[index:index+nItems]))
    index += nItems
    for u in users:
        userGamma[u] = theta[index:index+K]
        index += K
    for i in items:
        itemGamma[i] = theta[index:index+K]
        index += K

Similarly, our cost and derivative functions serve the same role as before, though their implementations are somewhat more complicated.

In [22]:
def inner(x, y):
    return sum([a*b for a,b in zip(x,y)])


def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item] + inner(userGamma[user], itemGamma[item])


def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d['customer_id'], d['product_id']) for d in dataset]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in users:
        cost += lamb*userBiases[u]**2
        for k in range(K):
            cost += lamb*userGamma[u][k]**2
    for i in items:
        cost += lamb*itemBiases[i]**2
        for k in range(K):
            cost += lamb*itemGamma[i][k]**2
    return cost


def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(dataset)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    dUserGamma = {}
    dItemGamma = {}
    for u in reviewsPerUser:
        dUserGamma[u] = [0.0 for k in range(K)]
    for i in reviewsPerItem:
        dItemGamma[i] = [0.0 for k in range(K)]
    for d in dataset:
        u,i = d['customer_id'], d['product_id']
        pred = prediction(u, i)
        diff = pred - d['star_rating']
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
        for k in range(K):
            dUserGamma[u][k] += 2/N*itemGamma[i][k]*diff
            dItemGamma[i][k] += 2/N*userGamma[u][k]*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
        for k in range(K):
            dUserGamma[u][k] += 2*lamb*userGamma[u][k]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
        for k in range(K):
            dItemGamma[i][k] += 2*lamb*itemGamma[i][k]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    for u in users:
        dtheta += dUserGamma[u]
    for i in items:
        dtheta += dItemGamma[i]
    return numpy.array(dtheta)

Again we optimize using our gradient descent library, and compare to a simple baseline.

In [23]:
MSE(alwaysPredictMean, labels) #Same as our previous baseline

2.371535478415058

In [24]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + # Initialize alpha
                                   [0.0]*(nUsers+nItems) + # Initialize beta
                                   [random.random() * 0.1 - 0.05 for k in range(K*(nUsers+nItems))], # Gamma
                             derivative, args = (labels, 0.001), maxfun = 10, maxiter = 10)

#Note the "maxfun = 10" and "maxiter = 10" this is because this function will go on for over 
# 20 iterations taking far too long to compute.

MSE = 2.4039771115383703
MSE = 2.867537580384821
MSE = 2.358389877235546
MSE = 2.3459233397944073
MSE = 2.2995547624492003
MSE = 2.1016366596113816
MSE = 2.081207562567891
MSE = 1.990495389205931
MSE = 1.9579272490317554
MSE = 1.9280840708546934
MSE = 1.9287580983240948


(array([ 3.65773097e+00, -2.98500410e-03,  3.64114602e-03, ...,
        -1.43250772e-03,  1.57282481e-03, -3.28605980e-04]),
 2.0278783274268504,
 {'grad': array([-8.86247270e-04,  2.83985269e-07, -2.67902629e-06, ...,
         -2.76174351e-06,  3.22165757e-06, -6.11126218e-07]),
  'task': b'STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT',
  'funcalls': 11,
  'nit': 8,
  'warnflag': 1})

Note finally that in the above exercise we only computed the ___training___ error of our model, i.e., we never confirmed that it works well on held-out (validation/testing) data!

## You're all done!

This weeks notebook was fairly simple (homework-wise), but the concepts were rather difficult. Next week you will start your capstone project, which will combine all 4 courses into a single assignment! Remember to use all your available resources when you start the project, including your previous notebooks!